In [3]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
tablemap={"coupon_emr_etl":"coupon"} #,
# 'insert' as kind,
#     '2020-08-01 23:59:59' as etldate,
#         0 as  etlindex,
#      a.*,
#     '2020' as year,
#     '' as month,
#      '01' as day
first=" select 'insert' as kind, '2021-01-01 23:59:59' as etldate,0 as  etlindex, "
end=" , '2021' as year,   '01' as month,  '01' as day from  "
def getTableColum(tableNm,b):
    colum=""
    for index in range(len(b)):
        colum=colum+setDef(b[index]["data_type"],b[index]["col_name"],tableNm)+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col,tableNm):
    if(tableNm=='t_auto_review_loan' and type[:7] == 'boolean'):
        return "cast ("+table_col+" as string ) as "+table_col
    else:
        return table_col
def make_col_temptable(databaseName,row):
    tableName=row["tableName"]
#     if tableName=='t_customer':
    tablecolum="desc " +databaseName+"."+tableName;
    tableSchema= spark.sql(tablecolum).collect()
    sql=first+ getTableColum(tableName,tableSchema)+end+databaseName+"."+tableName
    path="s3://rupiahplus-data-warehouse/stream/"+tablemap[databaseName]+"_etl/"+tableName+"/"
    spark.sql(sql).write.mode("append").partitionBy("year","month","day").orc(path)
if __name__ == "builtins":
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for databaseName in tablemap:
        databasesql="show tables in "+databaseName
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm) for tableNm in tablelist])
            for  future in futures_result[0]:
                print("aa")


VBox()

Exception in thread cell_monitor-3:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

aa
aa
aa

In [ ]:
t_coupon_change_log=spark.sql("""
    select
      'insert' as kind,
     '2020-03-04 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '03' as month,
     '04' as day
    from `coupon_emr_etl`.t_coupon_change_log
    where  id< 595640
""")
t_coupon_change_log.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/coupon_etl/t_coupon_change_log/")


In [1]:
t_coupon_record=spark.sql("""
    select
      'insert' as kind,
     '2020-03-04 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '03' as month,
     '04' as day
    from `coupon_emr_etl`.t_coupon_record

""")
t_coupon_record.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/coupon_etl/t_coupon_record/")
#     where  id< 274996

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1596076399298_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
t_coupon_store=spark.sql("""
    select
      'insert' as kind,
     '2020-03-04 23:59:59' as etldate,
        0 as  etlindex,
     *,
    '2020' as year,
    '03' as month,
     '04' as day
    from `coupon_emr_etl`.t_coupon_store
    where id < 274996 
""")
t_coupon_store.write.mode("append").partitionBy("year","month","day").orc("s3://rupiahplus-data-warehouse/stream/coupon_etl/t_coupon_store/")
# where  id< 233791

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…